In [ ]:
## MOUNT GOOGLE DRIVE

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## PACKAGE IMPORTS

import pandas as pd
import os
import numpy as np

In [ ]:
## IMPORT DATA FILES

THT_df = pd.read_csv('/content/gdrive/My Drive/ECE 225A Project (Walkability)/data/THT_Data_508_cleaned.csv') # transportation
obesity_df = pd.read_csv('/content/gdrive/My Drive/ECE 225A Project (Walkability)/data/Obesity.csv')
binge_drinking_df = pd.read_csv('/content/gdrive/My Drive/ECE 225A Project (Walkability)/data/Binge_Drinking.csv')
mental_health_df = pd.read_csv('/content/gdrive/My Drive/ECE 225A Project (Walkability)/data/Mental_health.csv')


In [ ]:
# REMOVE NO DATA ROWS
def remove_empty_rows(df):
    df = df.replace("", np.nan)
    return df.dropna(axis='rows')

# AGGREGATE CITY DATA
def aggregate_cities(df):
    datacol = df.columns[5]
    cleaned_list = []

    df_splits = [v for k, v in df.groupby('CityName')]
    for df_split in df_splits:
      data_row = df_split.iloc[0]  # get first row of dataframe

      tot_pop = np.sum(df_split['PopulationCount'])
      w_avg = np.sum(df_split[datacol] * df_split['PopulationCount'])/tot_pop

      data_row[datacol] = w_avg
      data_row['PopulationCount'] = tot_pop
      cleaned_list.append([data_row['citystate'], data_row[datacol]])

    return pd.DataFrame(cleaned_list, columns = ['citystate', datacol])



In [ ]:
city = 'San Francisco, CA'

if (obesity_df['citystate'] == city).any():
    print(f'{city} is in obesity file before modifications')

if (binge_drinking_df['citystate'] == city).any():
    print(f'{city} is in binge drinking file before modifications')

if (mental_health_df['citystate'] == city).any():
    print(f'{city} is in mental health file before modifications')

San Francisco, CA is in obesity file before modifications
San Francisco, CA is in binge drinking file before modifications
San Francisco, CA is in mental health file before modifications


In [ ]:
## CLEAN FILES

obesity_df = remove_empty_rows(obesity_df)
obesity_cleaned_df = aggregate_cities(obesity_df)

binge_drinking_df = remove_empty_rows(binge_drinking_df)
binge_drinking_cleaned_df = aggregate_cities(binge_drinking_df)

mental_health_df = remove_empty_rows(mental_health_df)
mental_health_cleaned_df = aggregate_cities(mental_health_df)

In [ ]:
## MERGE DFS
combined_df = obesity_cleaned_df.set_index('citystate').combine_first(binge_drinking_cleaned_df.set_index('citystate')).reset_index()
combined_df = combined_df.set_index('citystate').combine_first(mental_health_cleaned_df.set_index('citystate')).reset_index()
combined_df = remove_empty_rows(combined_df)

combined_df.to_csv('/content/gdrive/My Drive/ECE 225A Project (Walkability)/data/combined_health.csv')
